Repovizz2 DataPack Upload Example
=================================

This notebook demonstrates basic datapack operations (upload and modify)

We will use here the repovizz2 client library to ease the authentication, but it is basically a standard OAuth2 authentication.

In [ ]:
from repovizz2 import RepoVizzClient

We will be using those modules:

In [ ]:
import json
# This is for the notebook
from IPython.display import IFrame, display

Now, some global parameters to connect to repovizz2

In [ ]:
CLIENT_ID = "1d77f100-dab1-4e99-84ae-3bf36584aafc"
CLIENT_SECRET = "031a141c-65fe-4143-9884-e637128ccdbc"


Repovizz2 authentication
-----------------------

We authenticate the client (this script) against repoVizz2 using a registered user. For more details on the authentication system see [repoVizz2 authentication](Authentication.ipynb).


This will create an iframe in the notebook to authenticate the client, but you can open a new browser to do so, if you want

In [ ]:
repovizz_client = RepoVizzClient(client_id=CLIENT_ID,client_secret=CLIENT_SECRET)
authorization_url = repovizz_client.start_auth()
repovizz_client.start_server(async=True)

# print authorization_url # and click on it
# - or -
# import webbrowser
# webbrowser.open(authorization_url)
# - or -
display(IFrame(authorization_url,800,600))

In [ ]:
repovizz_client.finish_auth()

Retrieving identity data
------------------------

We first get info about our identity. We can use `/api/v1.0/user` to get json information about the current logged user.

In [ ]:
myself = repovizz_client.get("/api/v1.0/user")
username = myself['username']
userid = myself['id']
print myself

Uploading a Datapack
--------------------

To upload a datapack:

* Upload the Datapack structure
* Upload its data files

Uploading the structure
-----------------------

Now we upload the datapack (without the data). The result of the call will include the upload datapack (item), including its ID. To do so, we post to `/api/v1.0/datapacks` with a new datapack using json.

In [ ]:
datapack_json = json.load(open('MYO_example_datapack/MYO_example_datapack.json'))
result = repovizz_client.post(
    "/api/v1.0/datapacks",
    json={
        'structure': datapack_json,
        'name': datapack_json['info']['name'],
        'owner': userid,
    },
)
datapack = result['item']

Listing the data files to upload
--------------------------------

Datapacks usually refer to external material, such as `csv` files, videos or audio. Those were'nt uploaded to the server yet. Instead, reference to their filenames were used as handles.

To upload all the real assets, first we can ask for all the data files referenced in the datapack using `/api/v1.0/datapacks/{datapack_id}/content`. This returns us a list of the referenced filenames.

In [ ]:
datapack_id = datapack['id']
data_in_datapack = repovizz_client.get('/api/v1.0/datapacks/{}/content'.format(datapack_id)).keys()
data_in_datapack

Uploading the data files
------------------------

For each of those files, we have to send the actual content to be saved in the server. We can do that by posting to `/api/v1.0/datapacks/{datapack_id}/content/{content_id}` with the actual file attached.

Repovizz2 will substitute the content identifier with a new one, using the format `repovizz:{hashcode}`, meaning that the contents of this file can be found in the repovizz2 server, with the _hashcode_ as its handle.

In [ ]:
for fname in data_in_datapack:
    print repovizz_client.post(
        '/api/v1.0/datapacks/{}/content/{}'.format(datapack_id,fname),
        files={fname:open('MYO_example_datapack/'+fname)}
    )


Checking that everything has been properly uploaded
---------------------------------------------------

We can now check that indeed we changed the references to the contents in the datapack by quering again `/api/v1.0/datapacks/{datapack_id}/content`.

In [ ]:
data_in_datapack = repovizz_client.get('/api/v1.0/datapacks/{}/content'.format(datapack_id)).keys()
data_in_datapack

With this step completed, we concluded uploading the datapack to repovizz2. See [Retrieval](Retrieval.ipynb) for the next steps.